# **Subqueries**

A **subquery** is a query **nested** inside of another query, the advantage of doing this is that it allows
- *Comparing* grouped data to summarized values
- *Reshaping* of data
- *Combining* of data where a join or set operation is not possible

In [1]:
%load_ext sql
%sql sqlite:///databases/world.db

## Subqueries Inside `SELECT`

If a query returns a single column, then that column can be used in a new table.

In [2]:
%%sql
SELECT 
    DISTINCT continent, 
    (
        SELECT COUNT(*)
        FROM monarchs
        WHERE states.continent = monarchs.continent
    ) AS monarch_count
FROM states
ORDER BY monarch_count DESC
LIMIT(5)

 * sqlite:///databases/world.db
Done.


continent,monarch_count
Asia,2
Europe,2
Oceania,0
South America,0
Africa,0


Things to keep in mind
- Return a single value (to be compared with each row value of the main table)
- Preferably apply the same filter to  the main and sub query 


In [3]:
%sql sqlite:///databases/football.db

In [4]:
%%sql 
SELECT 
    date,
    (home_goal + away_goal) AS goals,
    ROUND((home_goal + away_goal) - (
        SELECT AVG(home_goal+away_goal) 
        FROM matches
        WHERE season = "2011/2012"
    ), 2) AS diff
FROM matches
WHERE season = "2011/2012"
LIMIT(5)

 * sqlite:///databases/football.db
   sqlite:///databases/world.db
Done.


date,goals,diff
2011-07-29T00:00:00.000,3,0.28
2011-07-30T00:00:00.000,2,-0.72
2011-07-30T00:00:00.000,4,1.28
2011-07-30T00:00:00.000,1,-1.72
2011-07-30T00:00:00.000,0,-2.72


## Subqueries Inside `FROM`

By including multiple tables in the `FROM` operator, the same effect as an `INNER JOIN` can be achieved. In practice the `INNER JOIN` operator is preferred due to its clarity and simply because of convention. 

In [5]:
%sql sqlite:///databases/world.db

In [6]:
%%sql 

SELECT pm.country, pm.continent, pm.prime_minister, p.president 
FROM prime_ministers AS pm 
INNER JOIN presidents AS p ON pm.country = p.country
LIMIT(5);

-- is equivalent to --

SELECT pm.country, pm.continent, pm.prime_minister, p.president 
FROM 
    prime_ministers AS pm, 
    presidents AS p 
WHERE pm.country = p.country
LIMIT(5);

   sqlite:///databases/football.db
 * sqlite:///databases/world.db
Done.
Done.


country,continent,prime_minister,president
Egypt,Africa,Sherif Ismail,Abdel Fattah el-Sisi
Portugal,Europe,Antonio Costa,Marcelo Rebelo de Sousa
Vietnam,Asia,Nguyen Xuan Phuc,Tran Dai Quang
Haiti,North America,Jack Guy Lafontant,Jovenel Moise


Instead of using tables in the `FROM` operation, a subquery can be used. In the example below the  `DISTINCT` operator can be used in order to remove the duplicate values, making the operation equivalent to the `INTERSECT` operator instead of an `INNER JOIN`. 

In [7]:
%%sql 
SELECT continent, MAX(indep_year) AS most_recent
FROM states
GROUP BY continent

   sqlite:///databases/football.db
 * sqlite:///databases/world.db
Done.


continent,most_recent
Africa,1922
Asia,1984
Europe,1905
North America,1804
Oceania,1901
South America,1828


In [8]:
%%sql 
SELECT DISTINCT monarchs.continent, sub.most_recent
FROM 
    monarchs, 
    (
        SELECT continent, MAX(indep_year) AS most_recent
        FROM states
        GROUP BY continent
    ) AS sub
WHERE monarchs.continent = sub.continent
ORDER BY monarchs.continent
    

   sqlite:///databases/football.db
 * sqlite:///databases/world.db
Done.


continent,most_recent
Asia,1984
Europe,1905


## Subqueries Inside `WHERE`



In [9]:
%sql sqlite:///databases/football.db

The data can be filtered using `WHERE` based on a value calculated in a subquery. 

In [10]:
%%sql

SELECT 
    date,
    hometeam_id,
    awayteam_id,
    home_goal,
    away_goal
FROM matches
WHERE 
    season = '2012/2013'
    AND home_goal > (SELECT AVG(home_goal) FROM matches)
LIMIT(5)

 * sqlite:///databases/football.db
   sqlite:///databases/world.db
Done.


date,hometeam_id,awayteam_id,home_goal,away_goal
2012-07-28T00:00:00.000,9993,9994,2,4
2012-07-28T00:00:00.000,9998,1773,5,2
2012-07-28T00:00:00.000,8203,9986,4,2
2012-07-28T00:00:00.000,8342,8475,3,1
2012-07-29T00:00:00.000,9991,9989,2,0


Or the data can be filtered using `WHERE` based on an entire column returned by in a subquery. 

In [11]:
%%sql
SELECT team_long_name, team_short_name
FROM teams
WHERE team_api_id IN (SELECT hometeam_id FROM matches WHERE country_id=15722)
LIMIT(5)

 * sqlite:///databases/football.db
   sqlite:///databases/world.db
Done.


team_long_name,team_short_name
Wisła Kraków,WIS
Polonia Bytom,GOR
Ruch Chorzów,CHO
Legia Warszawa,LEG
P. Warszawa,PWA


## Nested Subqueries

As the name suggests, a **nested subquery** is a subquery within a subquery. In the example below a sqlite function `strftime`, this is equivalent to the `EXTRACT` method in PostgreSQL

In [12]:
%%sql
SELECT 
    strftime('%m', date) AS month,
    SUM(m.home_goal + m.away_goal) AS total_goals,
    ROUND(SUM(m.home_goal + m.away_goal) - (
        SELECT AVG(goals)
        FROM (
            SELECT 
                strftime('%m', date) AS month,
                SUM(home_goal + away_goal) AS goals
            FROM matches
            GROUP BY month
        ) AS s
    ), 2) AS diff
FROM matches AS m
GROUP BY month
LIMIT(5)

 * sqlite:///databases/football.db
   sqlite:///databases/world.db
Done.


month,total_goals,diff
01,2744,-178.67
02,3711,788.33
03,3902,979.33
04,3978,1055.33
05,3003,80.33


## Commont Table Expressions (CTE)

As can be seen in the previous example, these subquerries can become very large and messy.  In order to improve **readability** of a query, simple subqueries can be factored out of the main query using a **Common Table Expression (CTE)**. 

It should be noted that the CTE only exists within the context of the current query, if you want other queries to reference your subquerry you are better off using a virtual table (`VIEW`)

In [13]:
%%sql

WITH s AS (
    SELECT 
        strftime('%m', date) AS month,
        SUM(home_goal + away_goal) AS goals
    FROM matches
    GROUP BY month
)

SELECT 
    strftime('%m', date) AS month,
    SUM(m.home_goal + m.away_goal) AS total_goals,
    ROUND(SUM(m.home_goal + m.away_goal) - (SELECT AVG(goals) FROM s), 2) AS diff
FROM matches AS m
GROUP BY month
LIMIT(5);

 * sqlite:///databases/football.db
   sqlite:///databases/world.db
Done.


month,total_goals,diff
01,2744,-178.67
02,3711,788.33
03,3902,979.33
04,3978,1055.33
05,3003,80.33


## Summary

### Joining Data
**Join Operation**		
- *Advantage*	This is the general/universal way of combining data from two tables
- *Limitation*	You can’t join two separate columns in a table to a single column in another table

**Correlated Subqueries**
- *Advantage*	Allows matching data from different columns (avoids limitation of join operation)
- *Limitation*	It is computationally expensive

### Multi-Step Querries

**Nested Queries**  
- *Advantage*:	Allows computation in multiple steps
- *Limitation*:	Reduced readability/organization

**CTE’s**  
- *Advantage*:	Increased readability/organization, allows computation in multiple steps
- *Limitation*:	More bookkeeping when writing the code